In [4]:
!pip install -q noisereduce
!pip install -q git+https://github.com/TenzinGayche/num2tib.git

In [1]:
import librosa
import numpy as np
import torch
import pyewts
import noisereduce as nr
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from num2tib.core import convert
from num2tib.core import convert2text
import re
from pathlib import Path
from IPython.display import Audio, display

In [2]:
Path.cwd()

PosixPath('/root/speech_t5_tts')

In [8]:
converter = pyewts.pyewts()
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model_checkpoint = "./TTS_st5_25102024/checkpoint-9000"
model = SpeechT5ForTextToSpeech.from_pretrained(model_checkpoint)
model.to('cuda')
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [9]:
def replace_numbers_with_convert(sentence, wylie=True):
    pattern = r'\d+(\.\d+)?'
    def replace(match):
        return convert(match.group(), wylie)
    result = re.sub(pattern, replace, sentence)
    
    return result

replacements = [
    ('_', '_'),
    ('*', 'v'),
    ('`', ';'),
    ('~', ','),
    ('+', ','),
    ('\\', ';'),
    ('|', ';'),
    ('╚',''),
    ('╗','')
]
def cleanup_text(text):
    for src, dst in replacements:
        text = text.replace(src, dst)
    return text

In [10]:
def predict(text):
    if len(text.strip()) == 0:
        return (16000, np.zeros(0).astype(np.int16))
    text = converter.toWylie(text)
    text=cleanup_text(text)
    text=replace_numbers_with_convert(text)
    inputs = processor(text=text, return_tensors="pt")
    # limit input length
    input_ids = inputs["input_ids"]
    input_ids = input_ids[..., :model.config.max_text_positions]
    speaker_embedding = np.load("./tests/female_2.npy")
    speaker_embedding = torch.tensor(speaker_embedding)
    speech = model.generate_speech(input_ids.to('cuda'), speaker_embedding.to('cuda'), vocoder=vocoder.to('cuda'))
    speech = nr.reduce_noise(y=speech.to('cpu'), sr=16000)
    return (16000, speech)

In [11]:
examples = [
    ["ད་དེ་ཚོ་འདི་བྱེད་དགོས་རེད་ ན་ཚ་ མ་ཡོང་སྔོན་ལ་ཁོ་རང་ལ་ཡང་ཁྱི་ཁོ་རང་ཁོ་ལ་ཡང་ཁབ་རྒྱག་ཡ་ཡོད་རེད། ཨུན་སྔོན་འགོག་དང་རཱབྷིསས་ཁབ་རྒྱག་ཡ་ཡོད་རེད་ད།", "Lhasa(female)"],
    ["སྟོབས་ཆེན་རྒྱལ་ཁབ་ཉི་ཤུའི་ལྷན་ཚོགས་ཐོག་ལ་རྒྱ་ནག་གཞུང་གིས་བོད་ནང་རིག་གཞུང་རྩ་གཏོར་ཀྱི་སྲིད་བྱུས་ཁག་དཔར་རིས་ཐོག་ནས་ལས་འགུལ་སྤེལ་བའི་སྐོར  འཇམ་དབྱངས་རྒྱ་མཚོ་ལགས་ཀྱིས་སྙན་སྒྲོན་གནང་གི་རེད།", "Lhasa(female)"],
    ["དངོས་གནས་ལབ་དགོས་རཱ་ད། མི་དབུལ་པོ་དེ་ཚོ་ལ་ག་རེ་ལབ་དགོས་རེད། སྦྱིན་པ་གཏང་ཡ་ཡོད་རཱ། ཨུན། དེ་འདྲ་གི་ལས་འགུལ་དེ་འདྲའི་མང་པོ་བརྩམས་ཀི་འདུག་བ། དེ་ཚོ་ཡང་ངས་ཚད་ལས་བརྒལ་བའི་ཡག་པོ་རེད་དྲན་གི་འདུག། ", "Lhasa(female)"],
    ["ཁོང་རྣམ་པ་ནི་སྤྱིར་བཏང་གི་གང་ཟག་ཅིག་མ་ཡིན་པར་མི་རབས་ནས་མི་རབས་རྒྱུད་པ་འཛིན་པའི་ནོར་བུ་ཡིན་ཞིང་། ", "Lhasa(female)"],
    ["ཨ་ལེ།  ཨེ་ནས་སྤྱིར་བཏང་ད་ང་ཚོ་ད་ལྟ་ཁྱེད་རང་གིས་དམིགས་ཡུལ་ད་གལ་ཆེན་པོ་བརྩིས་ནས།", "Lhasa(female)"],
    ["ཀུན་གླེང་གསར་འགྱུར། ༢༠༢༣་ལོའི་ཟླ་༩ ཚེས་༢༧ །", "Lhasa(female)"],
]

In [12]:
for text, _ in examples:
    rate, speech = predict(text)
    print(text)
    display(Audio(speech, rate=rate))

ད་དེ་ཚོ་འདི་བྱེད་དགོས་རེད་ ན་ཚ་ མ་ཡོང་སྔོན་ལ་ཁོ་རང་ལ་ཡང་ཁྱི་ཁོ་རང་ཁོ་ལ་ཡང་ཁབ་རྒྱག་ཡ་ཡོད་རེད། ཨུན་སྔོན་འགོག་དང་རཱབྷིསས་ཁབ་རྒྱག་ཡ་ཡོད་རེད་ད།


སྟོབས་ཆེན་རྒྱལ་ཁབ་ཉི་ཤུའི་ལྷན་ཚོགས་ཐོག་ལ་རྒྱ་ནག་གཞུང་གིས་བོད་ནང་རིག་གཞུང་རྩ་གཏོར་ཀྱི་སྲིད་བྱུས་ཁག་དཔར་རིས་ཐོག་ནས་ལས་འགུལ་སྤེལ་བའི་སྐོར  འཇམ་དབྱངས་རྒྱ་མཚོ་ལགས་ཀྱིས་སྙན་སྒྲོན་གནང་གི་རེད།


དངོས་གནས་ལབ་དགོས་རཱ་ད། མི་དབུལ་པོ་དེ་ཚོ་ལ་ག་རེ་ལབ་དགོས་རེད། སྦྱིན་པ་གཏང་ཡ་ཡོད་རཱ། ཨུན། དེ་འདྲ་གི་ལས་འགུལ་དེ་འདྲའི་མང་པོ་བརྩམས་ཀི་འདུག་བ། དེ་ཚོ་ཡང་ངས་ཚད་ལས་བརྒལ་བའི་ཡག་པོ་རེད་དྲན་གི་འདུག། 


ཁོང་རྣམ་པ་ནི་སྤྱིར་བཏང་གི་གང་ཟག་ཅིག་མ་ཡིན་པར་མི་རབས་ནས་མི་རབས་རྒྱུད་པ་འཛིན་པའི་ནོར་བུ་ཡིན་ཞིང་། 


ཨ་ལེ།  ཨེ་ནས་སྤྱིར་བཏང་ད་ང་ཚོ་ད་ལྟ་ཁྱེད་རང་གིས་དམིགས་ཡུལ་ད་གལ་ཆེན་པོ་བརྩིས་ནས།


ཀུན་གླེང་གསར་འགྱུར། ༢༠༢༣་ལོའི་ཟླ་༩ ཚེས་༢༧ །
